In [ ]:
import os
import sys

# Obtenir le répertoire de travail courant
current_dir = os.getcwd()

# Obtenir le répertoire parent de `maps` (qui est `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Ajouter `src` au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import pandas as pd
import numpy as np
import folium.features
import folium.plugins

from useful_methods.data_processing import extract_data
from useful_methods.ihm.plots import plot_graph
from useful_methods.neighbours_delaunay.graphs import delaunay_graph
from useful_methods.neighbours_delaunay.miscellaneous_for_neighbouring import mean_distance_to_NN
from useful_methods.neighbours_delaunay.enhanced_criteria import *

In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')

In [ ]:
df_extracted = extract_data(df, provider='Orange', region='Nouvelle-Aquitaine', techno='4g')

In [ ]:
mean_distance_params = {
    ']0, 1] km': {'colour': '#030464', 'angle': 40, 'distance': 2},
    ']1, 2] km': {'colour': '#069AF3', 'angle': 30, 'distance': 5},
    ']2, 4] km': {'colour': '#02D4BB', 'angle': 20, 'distance': 10},
    ']4, inf] km': {'colour': '#0DBF75', 'angle': 20, 'distance': 15},
}

## With Delaunay

In [ ]:
G, pos = delaunay_graph(df_extracted)

https://www.researchgate.net/figure/Pseudo-code-for-constructing-Gabriel-graphs_fig2_225256686

https://gist.github.com/jgostick/0fcc3dbd61c313384f31909a8e9cf219?permalink_comment_id=4245005

In [ ]:
def gabriel_graph(G, df):
    gab_G = deepcopy(G)

    coordsXY = df[['x','y']]

    for edge in G.edges:
        pt1 = edge[0]
        pt2 = edge[1]

        middle_point = (coordsXY.loc[pt1] + coordsXY.loc[pt2])/2

        neigh = NearestNeighbors(radius=np.sqrt(np.sum((coordsXY.loc[pt1] - coordsXY.loc[pt2])**2, axis=0))/2)
        neigh.fit(coordsXY)

        if(len(coordsXY.iloc[neigh.radius_neighbors([middle_point], sort_results=True)[1][0][:-2]].index)>0):
            gab_G.remove_edges_from([edge])

    return gab_G

In [ ]:
gab_G = gabriel_graph(G, df_extracted)

In [ ]:
gab_G = angle_criterion_enhanced(gab_G, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)

In [ ]:
max = 0
for bs_id in pos.keys():
    tmp = len(G.edges(bs_id))
    if tmp > max:
        max = tmp

max

In [ ]:
mean_distances = mean_distance_to_NN(df_extracted[['x', 'y']], n_neighbours=3) # 3 to have more neighbours

In [ ]:
G_tot = distance_criterion_enhanced(G, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
G_tot = angle_criterion_enhanced(G_tot, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
G_tot = quadrant_criterion_enhanced(G_tot, pos)

In [ ]:
plot_graph(G_tot, pos)

In [ ]:
len(G_tot.edges)

## Without Delaunay

In [ ]:
G_dist = nx.Graph()
nodes = df_extracted.index
G_dist.add_nodes_from(nodes)

## Nouvelle idée : prendre les 7 plus proches voisins pour créer le graphe

In [ ]:
coordsXY = df_extracted[['x', 'y']]
n_neighbours = 15

nbrs = NearestNeighbors(n_neighbors=n_neighbours+1).fit(coordsXY)
neighs = nbrs.kneighbors(coordsXY, return_distance=False)

# data_mean_dist = np.mean(distances[:, 1:]/1000, axis=1)  # we exclude the first element (distance to ourself is 0)
# mean_distances_other = pd.Series(data=data_mean_dist, index=coordsXY.index)

In [ ]:
neighs

In [ ]:
# for i in range(len(neighs)):
#     neighs = np.where(neighs==i, neighs, coordsXY.index[i])
for row in range(len(neighs)):
    for col in range(len(neighs[row])):
        neighs[row, col] = coordsXY.index[neighs[row, col]]

In [ ]:
for row in neighs:
    bs_id = row[0]
    for bs_id_neigh in row[1:]:
        edge = [bs_id, bs_id_neigh]
        G_dist.add_edges_from([edge])

for node in tqdm(pos.keys(), desc="nodes - distance"):
    max_distance = mean_distance_choice(node, mean_distances, mean_distance_params, 'distance')

    for neigh_ind in coordsXY.iloc[nbrs.radius_neighbors([coordsXY.loc[node]], radius=max_distance*1000, sort_results=True)[1][0][1:]].index:
        edge = [node, neigh_ind]
        G_dist.add_edges_from([edge])

In [ ]:
G_dist_tot = distance_criterion_enhanced(G_dist, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
G_dist_tot = angle_criterion_enhanced(G_dist_tot, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
# G_dist_tot = angle_criterion_enhanced(G_dist_tot, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
G_dist_tot = quadrant_criterion_enhanced(G_dist_tot, pos)
G_dis = distance_criterion_enhanced(G_dist, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
G_ang = angle_criterion_enhanced(G_dist, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
# G_ang = angle_criterion_enhanced(G_ang, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
G_qua = quadrant_criterion_enhanced(G_dist, pos)

In [ ]:
plot_graph(G_dist, pos)

In [ ]:
len(G_dist.edges)

## On a map

In [ ]:
def add_graph_edges(G_base: nx.Graph, G: nx.Graph, pos: dict, fg: folium.FeatureGroup, colour: str):
    for edge in G_base.edges:
        stations = []
        if(not(edge in G.edges)):
            stations.append(pos[edge[0]])
            stations.append(pos[edge[1]])

            folium.PolyLine(np.array(stations), color=colour, weight=2.5, opacity=1).add_to(fg)

In [ ]:
def addLegend(map, labelsToColors):
    legend_html = f'''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 120px; 
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color: white;
                "><strong>Legend</strong> <br>
    '''
    for label in labelsToColors.keys():
        legend_html += f"\n&nbsp; <span style='color:{labelsToColors.get(label).get('colour')}'>&#9632;</span> {label} <br>\n"
    legend_html += '</div>'
    map.get_root().html.add_child(folium.Element(legend_html))

In [ ]:
map = folium.Map(location=np.mean(df_extracted[['latitude','longitude']], axis=0), zoom_start=8.5, tiles="Cartodb Positron")
edges = folium.FeatureGroup(f"Edges ({len(G_dist.edges)})", show=False).add_to(map)
edges_gab = folium.FeatureGroup(f"Edges - Gabriel Graph ({len(gab_G.edges)})", show=True).add_to(map)
edges_dis = folium.FeatureGroup(f"Edges - distance ({len(G_dis.edges)})", show=False).add_to(map)
edges_ang = folium.FeatureGroup(f"Edges - angle ({len(G_ang.edges)})", show=False).add_to(map)
edges_qua = folium.FeatureGroup(f"Edges - quadrant ({len(G_qua.edges)})", show=False).add_to(map)
edges_tot = folium.FeatureGroup(f"Edges - total ({len(G_dist_tot.edges)})", show=False).add_to(map)
points = folium.FeatureGroup(f"Points ({len(G_dist)})").add_to(map)

add_graph_edges(G_dist, nx.Graph(), pos, edges, colour="lightblue")
add_graph_edges(gab_G, nx.Graph(), pos, edges_gab, colour="lightgreen")
add_graph_edges(G_dist, G_dis, pos, edges_dis, colour="red")
add_graph_edges(G_dist, G_ang, pos, edges_ang, colour="orange")
add_graph_edges(G_dist, G_qua, pos, edges_qua, colour="green")
add_graph_edges(G_dist_tot, nx.Graph(), pos, edges_tot, colour="#AAA662")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    color = mean_distance_choice(ind, mean_distances, mean_distance_params, 'colour')
    points.add_child(folium.CircleMarker(location=[latitude, longitude], color=color, radius=2.5, popup=f"commune : {df_extracted.loc[ind, 'nom_com']}\nmean_dist={mean_distances.get(ind)}", fillOpacity=1, fill=True))
folium.LayerControl().add_to(map)

addLegend(map, mean_distance_params)

map.save("../../out/maps/distance_graph_comp_neighbour_v2.html")

map = folium.Map(location=np.mean(df_extracted[['latitude','longitude']], axis=0), zoom_start=8.5, tiles="Cartodb Positron")
edges_tot = folium.FeatureGroup(f"Edges - total ({len(G_tot.edges)})").add_to(map)
edges_other = folium.FeatureGroup(f"Edges - other ({len(G_dist.edges)})").add_to(map)
points = folium.FeatureGroup(f"Points ({len(G)})").add_to(map)

add_graph_edges(G_dist, nx.Graph(), pos, edges_other, colour="orange")
add_graph_edges(G_tot, nx.Graph(), pos, edges_tot, colour="#AAA662")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    color = mean_distance_choice(ind, mean_distances, mean_distance_params, 'colour')
    points.add_child(folium.CircleMarker(location=[latitude, longitude], color=color, radius=2.5, popup=f"mean_dist={mean_distances.get(ind)}"))
folium.LayerControl().add_to(map)

addLegend(map, mean_distance_params)

map.save("../../out/maps/neighbourhood_nextGen.html")